In [1]:
import csv
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

#from sklearn.linear_model import LogisticRegression
#from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.naive_bayes import GaussianNB
#from sklearn.linear_model import Perceptron
#from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
#from sklearn.decomposition import PCA
#from sklearn import preprocessing 

In [2]:
# the task is to categorize user given info (bd, city, registered_via, registration duration) 
# and songs (via language, genre_ids, )... such that the later given user can be categorized and 
# we can predict whether the user will like a given song or not.
# One can choose to output predict_proba() for individual classifier and read out probability to be
# classifies as 1 (=replay).

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
songs = pd.read_csv('songs.csv')

In [4]:
#print(len(train))
#print(len(test))
#len(train[train.index.duplicated(keep=False)])
# train has no duplicates

In [5]:
user = pd.read_csv('members.csv')

In [6]:
# let's fill up "bd" from user data alone. #
user_num = user.drop('msno',axis=1)

In [7]:
user_id = user['msno']

In [8]:
#df_train['bd'].unique()

In [9]:
user_num.loc[(user_num['gender'].isnull()),'gender']=0
user_num['gender'] = user_num['gender'].map({0:0,'female':1,'male':2}).astype(int)

In [10]:
from scipy.stats import spearmanr,pearsonr

In [11]:
user_num.loc[(user_num['bd']<0)|(user_num['bd']>100),'bd']=0

In [12]:
# let's convert this into day of year and year
user_num['expiration_date'] = pd.to_datetime(user_num['expiration_date'],format='%Y%m%d')
user_num['exp_year'] = user_num['expiration_date'].dt.year.astype(int)
user_num['exp_doy'] = user_num['expiration_date'].dt.dayofyear.astype(int)
user_num= user_num.drop('expiration_date',axis=1)
user_num['registration_init_time'] = pd.to_datetime(user_num['registration_init_time'],format='%Y%m%d')
user_num['reg_year'] = user_num['registration_init_time'].dt.year.astype(int)
user_num['reg_doy'] = user_num['registration_init_time'].dt.dayofyear.astype(int)
user_num = user_num.drop('registration_init_time',axis=1)

In [13]:
spearmanr(user_num['bd'],user_num['exp_doy'])
#pearsonr(user_num['registration_init_time'],user_num['expiration_date'])
#city:0.7847 / 0.5461
#gender:0.8713 /0.691
#registered_via:0.2274 /0.2266
#reg_init_time:-0.5262 / -0.452
#expiration_date:0.3371 / 0.119
# pearsonr and spearmanr has similar correlation values.
# let's drop gender
# let's predict bd based on other user data (#registration_init_time, registered_via and expiration_date)
# we use decision tree algorithm to fill in the missing bd

SpearmanrResult(correlation=0.060697900431217984, pvalue=1.8826363771224628e-29)

In [14]:
user_bd_fill = user_num.drop('city',axis=1)
user_bd_fill = user_bd_fill.drop('gender',axis=1)
user_bd = user_bd_fill['bd']

In [15]:
user_bd_null = user_bd_fill.loc[user_bd_fill['bd']==0]
user_bd_null = user_bd_null.drop('bd',axis=1)

In [16]:
user_bd_train = user_bd_fill.loc[user_bd_fill['bd']!=0]
Xuser_bd_train = user_bd_train.drop('bd',axis=1)
Yuser_bd_train = user_bd_train['bd']

In [17]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(Xuser_bd_train, Yuser_bd_train)
Y_pred = decision_tree.predict(user_bd_null)
acc_decision_tree = round(decision_tree.score(Xuser_bd_train, Yuser_bd_train) * 100, 2)
print(acc_decision_tree)

96.88


In [18]:
user_bd_null['bd'] = Y_pred

In [19]:
user_bd_fill = pd.concat([user_bd_null,user_bd_train])

In [20]:
# these can be used as cross validation set
user_bd_drop=user_num[user_num.bd!=0]

In [21]:
user_bd_fill['msno'] = user['msno']

In [22]:
song_info = pd.read_csv('song_extra_info.csv')

In [23]:
df_train = pd.merge(train,user_bd_fill,on='msno',how='left')
df_test = pd.merge(test,user_bd_fill,on='msno',how='left')

In [24]:
song_mg = pd.merge(songs,song_info,on='song_id',how='left')

In [25]:
df_train = pd.merge(df_train,song_mg,on='song_id',how='left')
df_test = pd.merge(df_test,song_mg,on='song_id',how='left')

In [26]:
#df_train.loc[(df_train['bd']<0)|(df_train['bd']>100),'bd']=0

In [27]:
# train does not miss an index. How did missing index happen in tidy_split?

In [28]:
# gender does not correlate much with othervariable : cannot be filled
user_gender_drop =  user_num[user_num.gender!=0]

In [29]:
spearmanr(user_gender_drop['gender'],user_gender_drop['reg_year'])
#bd-registered_via 0.2258
#bd-registration_init_time -0.332
#bd-expiration_date 0.128
# gender does not have significant correlation to any of the variables: we can drop gender? should we check it against song variables?

SpearmanrResult(correlation=0.01446214719150264, pvalue=0.081600505983868893)

In [30]:
#plt.hist(df_train['bd'])
# too many unknown bds : about 40 % : instead of dropping them, find correlation and fill them in!
# e.g. registered_via, registration_init_time, expiration_date-registration_init_time
# is it different from including them in the training data separately?
# it is, because it may induce false correlation if we simply decide all the unknown bds are 0s #

In [31]:
#plt.hist(user_bd_fill['bd'])

In [32]:
#df_train.loc[(df_train['gender'].isnull()),'gender']=0

In [33]:
#df_train['gender'] = df_train['gender'].map({0:0,'female':1,'male':2}).astype(int)

In [34]:
df_test.loc[df_test['genre_ids'].isnull(),'genre_ids']=0

In [35]:
df_train.loc[df_train['genre_ids'].isnull(),'genre_ids']=0

In [36]:
# a fast way to split elements with given (esp unusual) delimeter
def tidy_split(df, column, sep='|', keep=False):
    """
    Split the values of a column and expand so the new DataFrame has one split
    value per row. Filters rows where the column is missing.

    Params
    ------
    df : pandas.DataFrame
        dataframe with the column to split and expand
    column : str
        the column to split and expand
    sep : str
        the string used to split the column's values
    keep : bool
        whether to retain the presplit value as it's own row

    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns as `df`.
    """
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    return new_df

In [37]:
#df_train[['genre1','genre2','genre3']] = df_train['genre_ids'].apply(lambda x: pd.Series(x.split('|')))
#df_train[['genre_id']] = pd.DataFrame(df_train['genre_ids'].str.split('|').tolist()).head()
df_train= tidy_split(df_train, 'genre_ids', sep='|')
df_test= tidy_split(df_test, 'genre_ids', sep='|')
# we will train the models, using all the duplicates
# then we can average the target likelihood of duplicates to estimate the final target probability

In [38]:
# save the duplicated index to dataframe
test_idx = pd.DataFrame(df_test.index)
train_idx = pd.DataFrame(df_train.index)

In [39]:
# check what variable correlates with genre_ids
#plt.scatter(df_train['bd'],df_train['registered_via'])
#pd.crosstab(df_train['genre_cut'],df_train['gender'],normalize='columns')
#spearmanr(df_train['genre_ids'],df_train['gender'])
#somehow this correlation calculation (and crosstab) never finishes#
#is it because it is too big?#
# bd does play a role in preferred genre # song_length, language (a bit)
# expiration_date does!!! WHY????
# registered_via does a little
# city does a littel too
# registration_init_time not much
# gender does not seem to #
# are these gender - genre correlation at any statistical significance? how can I tell that? what is the standard deviation? #
# for some genres, mixed gender fraction is significantly outside either one of the gender: does it mean it is small number stats#

In [40]:
# are song language and user city strongly correlated? - some exclusions do exist
# is there a language strongly preferred in 

In [41]:
train_id = df_train['msno']
test_id = df_test['msno']

In [42]:
X_train = df_train.drop('msno',axis=1)
X_test = df_test.drop('msno',axis=1)
X_train = X_train.drop('target',axis=1)
X_combined = [X_train,X_test]
X_train = X_train.drop('lyricist',axis=1)
X_train = X_train.drop('composer',axis=1)
X_train = X_train.drop('artist_name',axis=1)
X_train = X_train.drop('song_id',axis=1)
X_train = X_train.drop('name',axis=1)
X_train = X_train.drop('isrc',axis=1)

In [43]:
X_test = X_test.drop('lyricist',axis=1)
X_test = X_test.drop('composer',axis=1)
X_test = X_test.drop('artist_name',axis=1)
X_test = X_test.drop('song_id',axis=1)
X_test = X_test.drop('name',axis=1)
X_test = X_test.drop('isrc',axis=1)

In [44]:
X_train.loc[X_train['language'].isnull(),'language']=0
X_test.loc[X_test['language'].isnull(),'language']=0

In [45]:
test_id = X_test['id']
X_test = X_test.drop('id',axis=1)

In [46]:
Y_train = pd.DataFrame(df_train['target'])

In [47]:
X_train.loc[X_train['source_system_tab'].isnull(),'source_system_tab']=0
X_train.loc[X_train['source_screen_name'].isnull(),'source_screen_name']=0
X_train.loc[X_train['source_type'].isnull(),'source_type']=0
X_train.loc[X_train['song_length'].isnull(),'song_length']=0

In [48]:
#X_test.isnull().any()

In [49]:
X_test.loc[X_test['source_system_tab'].isnull(),'source_system_tab']=0
X_test.loc[X_test['source_screen_name'].isnull(),'source_screen_name']=0
X_test.loc[X_test['source_type'].isnull(),'source_type']=0
X_test.loc[X_test['song_length'].isnull(),'song_length']=0

In [50]:
n_train = len(X_train)
n_test = len(X_test)

In [51]:
#X_test.loc[X_test['genre_ids'].isnull(),'genre_ids']=0

In [52]:
X_tgt = pd.concat([X_train,X_test])

In [53]:
X_tgt = pd.get_dummies(X_tgt,columns=['source_system_tab','source_screen_name','source_type']).astype(int)

In [54]:
X_train = X_tgt[:n_train]
X_test = X_tgt[n_train:]

In [55]:
X_test.isnull().any().sum()

0

In [56]:
X_train.isnull().any().sum()

0

In [57]:
len(X_test)

2705361

In [58]:
X_test.head(5)

,bd,exp_doy,exp_year,reg_doy,reg_year,registered_via,song_length,genre_ids,language,source_system_tab_0,...,source_type_listen-with,source_type_local-library,source_type_local-playlist,source_type_my-daily-playlist,source_type_online-playlist,source_type_radio,source_type_song,source_type_song-based-playlist,source_type_top-hits-for-artist,source_type_topic-article-playlist
0,19,261,2017,50,2016,7,224130,458,3,0,...,0,1,0,0,0,0,0,0,0,0
1,19,261,2017,50,2016,7,320470,465,3,0,...,0,1,0,0,0,0,0,0,0,0
2,38,329,2016,322,2016,4,315899,2022,17,0,...,0,0,0,0,0,0,0,1,0,0
3,30,120,2017,206,2007,9,285210,465,52,0,...,0,0,0,0,0,1,0,0,0,0
4,30,120,2017,206,2007,9,197590,873,-1,0,...,0,0,0,0,0,1,0,0,0,0


In [59]:
#decision_tree = DecisionTreeClassifier(max_depth=20)
#decision_tree.fit(X_train, Y_train)
#acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
#print (acc_decision_tree)
# depth 20 tree performs better than 10!
# implement another algorithm to do the ML

In [60]:
random_forest = RandomForestClassifier(n_estimators=20, max_depth=30)
random_forest.fit(X_train, Y_train)
#Y_pred = clf.predict(X_test)
#print clf.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
print (acc_random_forest)
# accuracy too high : indicates that it has high variance (overfitting) problem. 
# Remember even the best models in the competition performs about 70 % success on the test dataset.
# best performance so far : RF w/ depth 30 est 20 (minor improvement over depth 20)

/home/ijee/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


77.73


In [61]:
Y_pred = random_forest.predict_proba(X_test)

In [62]:
Y_pred = pd.DataFrame(Y_pred)
Y_pred.describe()

,0,1
count,2.705361e+06,2.705361e+06
mean,5.375910e-01,4.624090e-01
std,1.815491e-01,1.815491e-01
min,0.000000e+00,0.000000e+00
25%,3.968669e-01,3.377914e-01
50%,5.404358e-01,4.595642e-01
75%,6.622086e-01,6.031331e-01
max,1.000000e+00,1.000000e+00


In [63]:
Y_pred['index'] = test_idx
Y_pred = Y_pred.groupby('index').mean()
#Y_pred = Y_pred[~Y_pred.index.duplicated(keep='first')]

In [64]:
# modify this such that it won't discard, but average, the duplicates
#df_train = df_train[~df_train.index.duplicated(keep='first')]

In [65]:
# modify this such that it won't discard, but average, the duplicates
#df_test = df_test[~df_test.index.duplicated(keep='first')]

In [66]:
submission = pd.DataFrame({
            "id": test_id,
                    "target": Y_pred.iloc[:,1]
                        })
submission = submission.groupby('id').mean().reset_index()

submission.to_csv('submission.csv', index=False)

In [67]:
if len(test) !=len(submission):
    print('something is wrong')